In [15]:
import k3d
import numpy as np
from bmcs_shell.folding.assembly.wb_scanned_cell import WBScannedCell
from bmcs_shell.api import WBTessellation4PEx
from scipy.spatial import cKDTree
import pandas as pd
from openpyxl import load_workbook

In [16]:
# Set the reference geometry to be generated from the closed form kinematics

wb_shell = WBTessellation4PEx(
                         a=1000/4,
                         b = 1615/4, 
                         c = 645/4, 
                         e_x = 286/4,
                         gamma=0.683, # a value of gamma = 0.75 was estimated from normals, but a CAD comparison showed that 0.75 doesn't lead to closer geometry to the scanned 
                         n_phi_plus=2, # planned 5 
                         n_x_plus=2,  # planned 3
                         wireframe_width=5,
                        ##---- Trimming function works only in WBTessellation4P ----##
                         trim_half_cells_along_y=True,
                         trim_half_cells_along_x=True,
#                          align_outer_nodes_along_x=True,
)
# wb_shell.interact()
orig_I_Fi = np.copy(wb_shell.I_Fi_trimmed)
orig_X_Ia = np.copy(wb_shell.X_Ia_trimmed)
#orig_X_Ia, orig_I_Fi, len(orig_X_Ia), len(orig_I_Fi)

In [17]:
# Define the instance cell of the WBScannedCell class

modules = {'WB305': ('WB305_facets_points.obj', [[1, 0], [np.pi/2, np.pi/2]]),
            'WB306': ('WB306_facets_points.obj', [[1, 0], [np.pi/2, -np.pi/2]]),
            'WB307': ('WB307_facets_points.obj', [[1], [np.pi/2]]),
            'WB308': ('WB308_facets_points.obj', [[1, 2], [np.pi/2, np.pi]]),
            'WB309': ('WB309_facets_points.obj', [[0, 2], [np.pi/2, -np.pi/2]]),
            'WB310': ('WB310_facets_points.obj', [[1, 2], [np.pi/2, np.pi]]),
            'WB311': ('WB311_facets_points.obj', [[1], [np.pi/2]]),
            'WB312': ('WB312_facets_points.obj', [[1], [-np.pi/2]]),
            'WB313': ('WB307_facets_points.obj', [[1], [np.pi/2]]),
            'WB314': ('WB310_facets_points.obj', [[1, 2], [np.pi/2, np.pi]]),
            'WB315': ('WB311_facets_points.obj', [[1], [np.pi/2]]),
            'WB316': ('WB311_facets_points.obj', [[1], [np.pi/2]]),
           }


In [18]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree

# Placeholder for aggregated data
aggregated_data = {}

# Main loop through modules
for module_id, (file_name, axes_angles) in modules.items():
    cell = WBScannedCell(file_path=file_name, rotate_system=[axes_angles[0], axes_angles[1]])

    # Create a KDTree for the scan points
    tree = cKDTree(cell.O_crease_nodes_X_Na)

    # Find the nearest neighbor in `scan` for each point in `orig`
    distances, scan_indices = tree.query(orig_X_Ia)

    # Rearrange the scan array to match the original array's order
    scan_rearranged = cell.O_crease_nodes_X_Na[scan_indices]

    # Differences in x, y, z
    diffs = scan_rearranged - orig_X_Ia

    # Initialize a DataFrame
    columns = ["orig_index", "scan_index", "orig_x", "orig_y", "orig_z",
               "scan_x", "scan_y", "scan_z", "delta_x", "delta_y", "delta_z", "euclidean_distance"]
    df = pd.DataFrame(columns=columns)

    # Populate the DataFrame
    for i in range(len(orig_X_Ia)):
        df.loc[i] = [
            i,                           # orig_index
            scan_indices[i],             # scan_index
            *orig_X_Ia[i],               # orig_x, orig_y, orig_z
            *scan_rearranged[i],         # scan_x, scan_y, scan_z
            *diffs[i],                   # delta_x, delta_y, delta_z
            distances[i]                 # euclidean_distance
        ]
        #Save the DataFrame to an Excel file
        df.to_excel("WBScanCellEvaluation_"+module_id+".xlsx", index=False, engine='openpyxl')

        # Populate aggregated data for statistics
        if i not in aggregated_data:
            aggregated_data[i] = {
                'delta_x': [], 'delta_y': [], 'delta_z': [],
                'euclidean_distance': []
            }
        aggregated_data[i]['delta_x'].append(diffs[i, 0])
        aggregated_data[i]['delta_y'].append(diffs[i, 1])
        aggregated_data[i]['delta_z'].append(diffs[i, 2])
        aggregated_data[i]['euclidean_distance'].append(distances[i])

    # Save or use the DataFrame `df` as needed

# Calculate statistics for each node
stats_data = []
for node_index, data in aggregated_data.items():
    stats = {
        # Node index
        'node_index': node_index,
        
        # Means
        'mean_delta_x': np.mean(data['delta_x']),
        'mean_delta_y': np.mean(data['delta_y']),
        'mean_delta_z': np.mean(data['delta_z']),
        'mean_distance': np.mean(data['euclidean_distance']),
        
        # Standard Deviations
        'std_delta_x': np.std(data['delta_x']),
        'std_delta_y': np.std(data['delta_y']),
        'std_delta_z': np.std(data['delta_z']),
        'std_distance': np.std(data['euclidean_distance']),
        
        # Coefficient of Variation (COV)
        'cov_delta_x': np.std(data['delta_x']) / np.mean(data['delta_x']) if np.mean(data['delta_x']) != 0 else 0,
        'cov_delta_y': np.std(data['delta_y']) / np.mean(data['delta_y']) if np.mean(data['delta_y']) != 0 else 0,
        'cov_delta_z': np.std(data['delta_z']) / np.mean(data['delta_z']) if np.mean(data['delta_z']) != 0 else 0,
        'cov_distance': np.std(data['euclidean_distance']) / np.mean(data['euclidean_distance']) if np.mean(data['euclidean_distance']) != 0 else 0,
        
        # 5% Quantiles
        'quantile_5_delta_x': np.percentile(data['delta_x'], 5),
        'quantile_5_delta_y': np.percentile(data['delta_y'], 5),
        'quantile_5_delta_z': np.percentile(data['delta_z'], 5),
        'quantile_5_distance': np.percentile(data['euclidean_distance'], 5),
        
        # 95% Quantiles
        'quantile_95_delta_x': np.percentile(data['delta_x'], 95),
        'quantile_95_delta_y': np.percentile(data['delta_y'], 95),
        'quantile_95_delta_z': np.percentile(data['delta_z'], 95),
        'quantile_95_distance': np.percentile(data['euclidean_distance'], 95),
    }

    stats_data.append(stats)

# Create a DataFrame for the statistics
stats_df = pd.DataFrame(stats_data)

# Save the statistics DataFrame to an Excel file
stats_df.to_excel("WBScanCellEvaluation_CreaseNodeStatistics.xlsx", index=False, engine='openpyxl')

,node_index,mean_delta_x,mean_delta_y,mean_delta_z,mean_distance,std_delta_x,std_delta_y,std_delta_z,std_distance,cov_delta_x,...,cov_delta_z,cov_distance,quantile_5_delta_x,quantile_5_delta_y,quantile_5_delta_z,quantile_5_distance,quantile_95_delta_x,quantile_95_delta_y,quantile_95_delta_z,quantile_95_distance
0,0,-0.063736,-1.136990e-08,-3.476938e-08,0.164472,0.200067,6.706040e-08,4.185314e-08,0.130530,-3.138997,...,-1.203736,0.793630,-0.380436,-9.471551e-08,-1.192093e-07,0.017593,0.143921,1.096632e-07,0.000000e+00,0.380436
1,1,0.063736,-4.868101e-08,1.490116e-08,0.164472,0.200067,4.255339e-08,2.853355e-08,0.130530,3.138997,...,1.914854,0.793630,-0.143921,-1.097040e-07,-1.341105e-08,0.017593,0.380436,2.793968e-10,5.960464e-08,0.380436
2,2,-2.053190,6.009410e+00,-3.261941e+00,7.521575,1.089153,2.000217e+00,1.243432e+00,1.062075,-0.530469,...,-0.381194,0.141204,-3.529468,3.412811e+00,-5.054604e+00,6.348224,-0.237900,8.704428e+00,-1.878094e+00,9.009762
3,3,-5.683154,4.089696e+00,-3.468842e+00,8.021061,1.019524,1.583627e+00,9.527192e-01,1.083640,-0.179394,...,-0.274651,0.135099,-6.818787,1.969727e+00,-4.840668e+00,6.471340,-3.948853,6.157036e+00,-2.507834e+00,9.436056
4,4,6.324099,-3.985819e+00,-3.656459e+00,8.498550,0.911433,1.638016e+00,6.537433e-01,0.983118,0.144121,...,-0.178791,0.115681,4.992615,-6.607216e+00,-4.509373e+00,7.777410,7.478543,-2.289705e+00,-2.591682e+00,10.022287
5,5,1.676043,-6.612264e+00,-3.250149e+00,7.985426,1.720903,2.544990e+00,8.994960e-01,1.890809,1.026765,...,-0.276755,0.236783,-1.014789,-1.141088e+01,-4.191109e+00,6.475012,3.735626,-3.854034e+00,-1.884821e+00,11.701037
6,6,3.556465,1.549591e+00,-3.692331e+00,5.400345,1.033339,3.648841e-01,5.662983e-01,1.020234,0.290552,...,-0.153372,0.188920,2.227570,1.063678e+00,-4.472395e+00,4.136850,4.934677,2.037322e+00,-2.864228e+00,6.623293
7,7,-3.573091,-1.297203e+00,-2.934457e+00,4.825624,0.732521,2.861443e-01,5.983497e-01,0.866370,-0.205011,...,-0.203905,0.179535,-4.234535,-1.735988e+00,-3.838394e+00,3.372771,-2.326473,-9.436626e-01,-2.219542e+00,5.735883
8,8,-6.688360,2.420643e+00,-2.503192e+00,7.776801,0.861624,1.444861e+00,1.334194e+00,0.995542,-0.128824,...,-0.532997,0.128014,-7.795471,9.513641e-01,-4.940719e+00,6.606151,-5.535185,4.930012e+00,-1.055003e+00,9.609072
9,9,-8.816239,2.420643e+00,-2.503192e+00,9.643588,0.470527,1.444861e+00,1.334194e+00,0.970494,-0.053370,...,-0.532997,0.100636,-9.577192,9.513641e-01,-4.940719e+00,8.609582,-8.220556,4.930012e+00,-1.055003e+00,11.593485


In [19]:
plot = k3d.plot(name='Vectors')
cell.plot_O_crease_lines(plot, line_numbers=False)
cell.plot_O_basis(plot)
cell.plot_points(plot, orig_X_Ia, point_size=5, color=0x0000ff, plot_numbers=True)
cell.plot_points(plot, scan_rearranged, point_size=5, color=0x007777, plot_numbers=True)

/Users/gui/Documents/bmcs_folder/bmcs_virtual/lib/python3.12/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [20]:
# create a list to receive the nodes index that form a facets
facets_N_OS = [[6, 15, 13, 14],
                [6, 14, 12],
                [6, 2, 10],
                [0, 2, 3, 1],
                [1, 3, 7],
                [7, 3, 8],
                [7, 8, 9, 10],
                [7, 10, 12, 11],
                [7, 11, 5],
                [7, 5, 1],
                [1, 5, 4, 0],
                [0, 4, 6],
                [6, 4, 17],
                [6, 17, 16, 15]]

In [21]:
def calculate_normal(vertices):
    """
    Calculate the normal of a triangle or quadrilateral facet.
    """
    if len(vertices) == 3:  # Triangle
        v1, v2, v3 = vertices
        vec1 = v2 - v1
        vec2 = v3 - v1
        normal = np.cross(vec1, vec2)
    elif len(vertices) == 4:  # Quadrilateral
        v1, v2, v3, v4 = vertices
        vec1 = v2 - v1
        vec2 = v4 - v1
        vec3 = v3 - v2
        normal1 = np.cross(vec1, vec2)
        normal2 = np.cross(vec3, vec1)
        normal = (normal1 + normal2) / 2  # Average the normals
    else:
        raise ValueError("Facet must have 3 (triangle) or 4 (quadrilateral) vertices.")
    
    return normal / np.linalg.norm(normal)  # Normalize the vector

def calculate_angle(normal1, normal2):
    """
    Calculate the angle (in degrees) between two normal vectors.
    """
    cos_theta = np.clip(np.dot(normal1, normal2), -1.0, 1.0)
    angle_radians = np.arccos(cos_theta)
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees

# Example usage:
def calculate_normals_and_angle(indices, orig_points, scan_points):
    orig_vertices = np.array([orig_points[i] for i in indices])
    scan_vertices = np.array([scan_points[i] for i in indices])

    n_orig = calculate_normal(orig_vertices)
    n_scan = calculate_normal(scan_vertices)

    angle = calculate_angle(n_orig, n_scan)
    
    return n_orig, n_scan, angle

# Example inputs:
orig_points = np.array([
    [0.0, 0.0, 2], 
    [1.0, 3, 0.0], 
    [0.0, 1.0, 0.0], 
    [1.0, 1.0, 0.0]
])  # Example original points

scan_points = np.array([
    [0.0, 0.0, 0.1], 
    [1.0, 5, 0.1], 
    [0.0, 4, 0.1], 
    [1.0, 1.0, 0.1]
])  # Example scanned points

indices = [0, 1, 2]  # Example indices for a triangle facet

n_orig, n_scan, angle = calculate_normals_and_angle(indices, orig_points, scan_points)

print(f"Normal vector of orig: {n_orig}")
print(f"Normal vector of scan: {n_scan}")
print(f"Angle between normals: {angle:.2f} degrees")


Normal vector of orig: [-0.87287156  0.43643578  0.21821789]
Normal vector of scan: [0. 0. 1.]
Angle between normals: 77.40 degrees
